In [ ]:
import numpy as np
import pandas as pd

# Load Data

In [ ]:
from enfobench.datasets import ElectricityDemandDataset

ds = ElectricityDemandDataset("../data/electricity-demand")

In [ ]:
unique_ids = ds.metadata_subset.list_unique_ids()

In [ ]:
len(unique_ids)

In [ ]:
unique_id = unique_ids[0]
unique_id

In [ ]:
target, _, metadata = ds.get_data_by_unique_id(unique_id)

In [ ]:
target.info()

# Make external forecasts based on covariates

# Create Dataset

In [ ]:
from enfobench import Dataset

univariate_dataset = Dataset(
    target=target,
    metadata=metadata,
)

# Create Model

In [ ]:
from enfobench import AuthorInfo, ModelInfo, ForecasterType
from enfobench.evaluation.utils import create_forecast_index


class ExampleModel:
    def __init__(self, param1: int):
        self.param1 = param1

    def info(self):
        return ModelInfo(
            name="TestModel",
            authors=[AuthorInfo(name="Attila Balint")],
            type=ForecasterType.point,
            params={
                "param1": 1,
            },
        )

    def forecast(
        self,
        horizon: int,
        history: pd.DataFrame,
        past_covariates=None,
        future_covariates=None,
        metadata: dict | None = None,
        level=None,
        **kwargs,
    ):
        index = create_forecast_index(history, horizon)
        return pd.DataFrame(
            index=index,
            data={
                "yhat": np.full(horizon, fill_value=history["y"].mean()) + self.param1,
            },
        )

In [ ]:
model = ExampleModel(1)

# Perform crossvalidation on your model locally

In [ ]:
from enfobench.evaluation import cross_validate

crossval_df = cross_validate(
    model,
    univariate_dataset,
    start_date=pd.Timestamp("2022-01-01T10:00:00"),
    end_date=pd.Timestamp("2022-02-01T00:00:00"),
    horizon=pd.Timedelta("38 hours"),
    step=pd.Timedelta("1 day"),
)

In [ ]:
crossval_df.head()

In [ ]:
cutoff_date_to_plot = crossval_df.cutoff_date.unique()[0]
crossval_df.loc[crossval_df.cutoff_date == cutoff_date_to_plot].set_index("timestamp").drop(
    columns=["cutoff_date"]
).plot()

# Evaluate metrics

In [ ]:
from enfobench.evaluation import evaluate_metrics
from enfobench.evaluation.metrics import mean_absolute_error, mean_bias_error

In [ ]:
evaluate_metrics(crossval_df, metrics={"MAE": mean_absolute_error, "MBE": mean_bias_error})

In [ ]:
metrics = evaluate_metrics(
    crossval_df,
    metrics={"MAE": mean_absolute_error, "MBE": mean_bias_error},
    groupby="cutoff_date",
)
metrics

In [ ]:
metrics[["MAE", "MBE"]].plot()